In [28]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from math import log

In [24]:
set_type = {
    '영화명': str,
    '영화명(영문)': str,
    '제작연도': int,
    '제작국가': str,
    '장르': str,
    '감독': str,
    '배우': str,
    '평점': str,
    '줄거리': str
}

data = pd.read_csv('../data/movie_list.csv')
data.head(2)

,영화명,영화명(영문),제작연도,제작국가,장르,감독,배우,평점,줄거리,Unnamed: 9
0,가구야공주 이야기,The Tale of Princess Kaguya,2013.0,일본,"애니메이션,드라마,판타지",다카하타 이사오,"아사쿠라 아키,코라 켄고",8.18,깊은 산속 마을의 할아버지는 우연히 빛나는 대나무 속에서 여자 아이를 발견하고 집으...,NaN
1,가까이서 본 기차,Closely Watched Trains,1966.0,체코,"전쟁,드라마,코미디",이리 멘젤,"바클라프 네카르,지트카 벤도바,요제프 소므르",7.71,"2차 세계대전 말, 독일점령하에 있는 체코의 어느 작은 시골역. 이제 막 연수를 마...",NaN


In [25]:
data.shape

(8252, 10)

In [26]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('줄거리 열의 결측값의 수:', data['줄거리'].isnull().sum())

줄거리 열의 결측값의 수: 650


In [27]:
# 결측값을 빈 값으로 대체
data['줄거리'] = data['줄거리'].fillna('')

In [ ]:
# https://sikaleo.tistory.com/62